In [4]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# 크롬 드라이버 버전이 수시로 업데이트되는 관계로 드라이버 설치하여 사용
# 환경에 맞게 저장 경로 변경 필요
driver = webdriver.Chrome('D:/PJB/MyDatalab/Develop-Crawling/Crawling_MyDatalab/chromedriver.exe')
driver.maximize_window()

# Naver Blog URL List(확장자: .csv) 읽어오기 
# 환경에 맞게 경로 변경 필요
input_data = pd.read_csv('D:/PJB/MyDatalab/Develop-Crawling/Crawling_MyDatalab/Input-data/Naver_Blog_Content_URL_list.csv')
url = input_data['URL']
url_val = url.values
url_list = url_val.tolist()

title_list = []
publisher_list = []
date_list = []
neighbor_list = []
like_list = []
comment_list = []

#  읽어들인 url list에 있는 url 하나씩 실행하며 필요한 데이터 추출
for url in url_list[:]:
    driver.get(url)
    driver.switch_to.frame('mainFrame')
    
    # 제목
    try:
        title = driver.find_element_by_css_selector('.se-module.se-module-text.se-title-text')
    except:
        title = driver.find_element_by_css_selector('.pcol1')
    title_list.append(title.text)
    
    # 게시자
    publisher = driver.find_element_by_css_selector('.nick')
    publisher_list.append(publisher.text)
    
    # 게시일
    try:
        date = driver.find_element_by_css_selector('.se_publishDate.pcol2')
    except:
        date = driver.find_element_by_css_selector('.date.fil5.pcol2._postAddDate')
    date_list.append(date.text)
    
    # 이웃 수
    try:
        neighbor = driver.find_element_by_css_selector('#widget-stat > div > ul > li:nth-child(1) > em')
        neighbor_list.append(neighbor.text)
    except:
        try:
            driver.switch_to.frame('BuddyConnectIframe')
            driver.find_element_by_xpath('//*[@id="nc_frame1"]/ul/li[2]/a').click()
            time.sleep(1)
            neighbor = driver.find_element_by_css_selector('#nc_frame1 > div.wrap.bg_main > div.content.bg_main.tab2 > div.buddy_cnt > p > strong')
            neighbor_list.append(neighbor.text)
        except:
            neighbor_list.append('-')
    
    # 좋아요 수
    try:
        like = driver.find_element_by_css_selector('#floating_bottom > div > div > div.area_sympathy > a > div > span > em.u_cnt._count')
        if like.text == '':
            like_list.append('0')
        else:
            like_list.append(like.text)
    except:
        like_list.append('-')
    
    # 댓글 수
    try:
        comment = driver.find_element_by_css_selector('#commentCount')
        if comment.text == '':
            comment_list.append('0')
        else:
            comment_list.append(comment.text)
    except:
        comment_list.append('-')

df = {'제목': title_list, 'URL': url_list, '게시자': publisher_list, '이웃 수': neighbor_list, '게시일': date_list, '좋아요 수': like_list, '댓글 수': comment_list}
output_data = pd.DataFrame(df)
# 추출된 데이터 .csv 파일로 저장
# 환경에 맞게 저장 경로 변경 필요
output_data.to_csv("D:/PJB/MyDatalab/Develop-Crawling/Crawling_MyDatalab/Output-data/naver_blog_data/naver_blog_data.csv", index = False, encoding='utf-8-sig')